<h2>Pipeline chains together multiple steps so that the output each step is used as input to next step.

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,LabelEncoder 

<h2>Titanic without Pipeline

In [23]:
df = pd.read_csv(r"C:\Users\Admin\Downloads\train.csv")

In [24]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [29]:
df=df.drop(columns=['Cabin'])

In [32]:
df.drop(columns=['PassengerId','Name','Ticket'],inplace=True)

In [34]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [43]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [35]:
x_train , x_test , y_train , y_test = train_test_split(df.drop(columns='Survived',axis=1),df['Survived'],test_size=0.2,random_state=0)

In [36]:
x_test.shape,x_train.shape

((179, 7), (712, 7))

In [38]:
from sklearn.impute import SimpleImputer

si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

x_train_age =si_age.fit_transform(x_train[['Age']])
x_train_embarked=si_embarked.fit_transform(x_train[['Embarked']])

x_test_age =si_age.transform(x_test[['Age']])
x_test_embarked=si_embarked.transform(x_test[['Embarked']])

In [56]:
x_train.loc[:, 'Age'] = np.squeeze(x_train_age)  # Convert 2D array to 1D if needed
x_train.loc[:, 'Embarked'] = np.squeeze(x_train_embarked)

x_test.loc[:, 'Age'] = np.squeeze(x_test_age)
x_test.loc[:, 'Embarked'] = np.squeeze(x_test_embarked)

In [57]:
oe_sex = OneHotEncoder(drop='first', sparse_output=False)

oe_embarked = OneHotEncoder(drop='first', sparse_output=False)

oe_train_sex = oe_sex.fit_transform(x_train[['Sex']])
oe_train_embarked = oe_embarked.fit_transform(x_train[['Embarked']])

oe_test_sex = oe_sex.transform(x_test[['Sex']])
oe_test_embarked = oe_embarked.transform(x_test[['Embarked']])

KeyError: "None of [Index(['Sex'], dtype='object')] are in the [columns]"

In [52]:
sex_cols = oe_sex.get_feature_names_out(['Sex'])
embarked_cols = oe_embarked.get_feature_names_out(['Embarked'])

df_train_sex = pd.DataFrame(oe_train_sex, columns=sex_cols, index=x_train.index)
df_train_embarked = pd.DataFrame(oe_train_embarked, columns=embarked_cols, index=x_train.index)

df_test_sex = pd.DataFrame(oe_test_sex, columns=sex_cols, index=x_test.index)
df_test_embarked = pd.DataFrame(oe_test_embarked, columns=embarked_cols, index=x_test.index)

# Drop original columns and concatenate encoded ones
x_train = x_train.drop(['Sex', 'Embarked'], axis=1)
x_test = x_test.drop(['Sex', 'Embarked'], axis=1)

x_train = pd.concat([x_train, df_train_sex, df_train_embarked], axis=1)
x_test = pd.concat([x_test, df_test_sex, df_test_embarked], axis=1)

In [55]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier
clf.fit(x_train,y_train)

AttributeError: 'DataFrame' object has no attribute '_validate_params'